## PCR

----

### 1. Import Modules

In [1]:
import numpy as np
import pandas as pd
import cimcb as cb
from sklearn.model_selection import train_test_split

print('All packages successfully loaded')

Using TensorFlow backend.


All packages successfully loaded


### 2. Load data and peak sheet

In [2]:
home = 'data/' 
file = 'MTBLS93.xlsx' 

DataTable,PeakTable = cb.utils.load_dataXL(home + file, DataSheet='Data', PeakSheet='Peak') 

Loadings PeakFile: Peak
Loadings DataFile: Data
Data Table & Peak Table is suitable.
TOTAL SAMPLES: 2139 TOTAL PEAKS: 202
Done!


### 3. Extract X & Y

In [4]:
peaklist = PeakTable['Name']
DataTable2 = DataTable[(DataTable['Class']==0) | (DataTable['Class']==1)]
Y = DataTable2['Class'].values 

# Extract and scale the metabolite data from the DataTable
peaklist = PeakTable['Name']                           
XT = DataTable2[peaklist]                                    
XTlog = np.log(XT)                                          
XTscale = cb.utils.scale(XTlog, method='auto')              
XTknn = cb.utils.knnimpute(XTscale, k=3)                                      
                                         

### 4. Hyperparameters optimisation

In [8]:
# param_dict
#param_dict = {'n_components': [1,2,3,4,5,6]}                   
param_dict = {'n_components': [1,2,3,4,5,6,7,8,9,10,11,12]} 
param_dict = {'n_components': [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30]} 
param_dict = {'n_components': [10,20,30,40,50,60,70,80,90,100,110,120,130,140]}  # Do a next step.


# Initalise
cv = cb.cross_val.kfold(model=cb.model.PCR,                      
                                X=XTknn,                                 
                                Y=Y,                               
                                param_dict=param_dict,                   
                                folds=5,
                                n_mc=10)                                

# Run and plot
cv.run()  
cv.plot(metric='r2q2', ci=95)

Number of cores set to: 8
Running ...


100%|██████████| 140/140 [00:22<00:00,  4.94it/s]


Time taken: 0.41 minutes with 8 cores
Done!


Loading BokehJS ...

### 6. Get Model and Evaluate

In [12]:
# 6. Train, test and evaluate model

# Train model
model = cb.model.PCR(n_components=50)
model.train(XTknn, Y)
model.test(XTknn)

# Evaluate 
model.evaluate(cutoffscore=0.5, plot_median=False) 

Loading BokehJS ...

In [11]:
# Bootstrap evaluate

model.booteval(XTknn, Y, cutoffscore=0.5, bootnum=100) 

100%|██████████| 100/100 [00:06<00:00, 15.67it/s]


Loading BokehJS ...

### 7. Save table

In [13]:
home = 'tables/'
file = 'PCR_MTBLS93_NH.xlsx'

model.save_table(home + file)

Done! Saved table as tables/PCR_MTBLS93_NH.xlsx
